In [5]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [4]:
# Check available devices
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

NameError: name 'tf' is not defined

In [ ]:
# Step 1: Download and Load Dataset
path = kagglehub.dataset_download("gpiosenka/cats-in-the-wild-image-classification")
print("Path to dataset files:", path)

# Updated: Load images from class subfolders inside /valid
def load_images_from_subfolders(root_folder, img_size=(64, 64), max_images=1000):
    images = []
    for class_folder in os.listdir(root_folder):
        class_path = os.path.join(root_folder, class_folder)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(class_path, img_file)
                    try:
                        img = Image.open(img_path).convert("RGB").resize(img_size)
                        img_array = np.asarray(img) / 127.5 - 1  # Normalize to [-1, 1]
                        images.append(img_array)
                        if len(images) >= max_images:
                            return np.array(images)
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
    return np.array(images)

data_dir = os.path.join(path, "valid")
images = load_images_from_subfolders(data_dir, max_images=2000)
print("Loaded", images.shape[0], "images.")

In [ ]:
# Optional: Visualize sample
plt.imshow((images[0] + 1) / 2)
plt.axis('off')
plt.title("Sample Image")
plt.show()


In [ ]:
# Step 2: Define the Generator
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((8, 8, 256)),

        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model


In [ ]:
# Step 3: Define the Discriminator
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                      input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

In [ ]:
# Step 4: Define Loss Functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Step 5: Instantiate Models and Optimizers
generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:
# Step 6: Training Parameters
EPOCHS = 500000
BATCH_SIZE = 16
noise_dim = 100
seed = tf.random.normal([16, noise_dim])

train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(2000).batch(BATCH_SIZE)


In [ ]:


# Step 7: Define Training Loop
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(f"Epoch {epoch + 1}/{epochs} complete.")

        # Generate and show sample images every few epochs
        if (epoch + 1) % 10 == 0:
            generate_and_show(generator, seed)

# Step 8: Generate and Display Images
def generate_and_show(generator, seed):
    predictions = generator(seed, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i] + 1) / 2)
        plt.axis('off')

    plt.suptitle("Generated Images")
    plt.show()

# Step 9: Train the GAN
train(train_dataset, EPOCHS)


In [ ]:
generator.save("generator_model.h5")
